## selenium을 이용한 동적 크롤링 연습문제 : 네이버웹툰

### 네이버 웹툰 페이지에서 데이터 추출
- https://comic.naver.com/genre/bestChallenge.nhn  
- [전제베스트도전] 웹툰에서
    - 제목, 작가, 별점, 조회수 추출

### 문제1
- 한 페이지 크롤링
- https://comic.naver.com/genre/bestChallenge.nhn 페이지에서
- 현재 페이지의 [전제베스트도전] 웹툰에서
    - 제목, 작가, 별점, 조회수 추출해서
    - 데이터프레임 생성 : naver_webtoon_df

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
from urllib.request import urlopen
import requests
import bs4
import pandas as pd
import time

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 객체(모듈)

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [3]:
url = 'https://comic.naver.com/genre/bestChallenge.nhn'
driver.get(url)
time.sleep(2)

In [4]:
lis = driver.find_elements(By.CSS_SELECTOR, 'ul.BestChallengeView__challenge_list--sUqhh li')
name = lis[0].find_element(By.CLASS_NAME, 'ContentTitle__title--e3qXt')
name.text

writer = lis[0].find_element(By.CLASS_NAME, 'ContentAuthor__author--CTAAP')
writer.text

rate = lis[0].find_element(By.CLASS_NAME, 'Rating__star_area--dFzsb')
rate.text.split('\n')[1]

view = lis[0].find_element(By.CLASS_NAME, 'Rating__view_area--GQb_S')
view.text.split('\n')[1]

'덤덤씨 이야기'

'최덤덤'

'9.26'

'1.2만'

In [5]:
naver_webtoon_df = pd.DataFrame({
    '제목' : [],
    '작가' : [],
    '별점' : [],
    '조회수' : []
})

for li in lis:
    name = li.find_element(By.CLASS_NAME, 'ContentTitle__title--e3qXt').text
    writer = li.find_element(By.CLASS_NAME, 'ContentAuthor__author--CTAAP').text
    rate = li.find_element(By.CLASS_NAME, 'Rating__star_area--dFzsb').text.split('\n')[1]
    view = li.find_element(By.CLASS_NAME, 'Rating__view_area--GQb_S').text.split('\n')[1]
    
    df = pd.DataFrame({
        '제목' : [name],
        '작가' : [writer],
        '별점' : [rate],
        '조회수' : [view]
    })
    
    naver_webtoon_df = pd.concat([naver_webtoon_df, df], axis=0, ignore_index=True)

naver_webtoon_df

,제목,작가,별점,조회수
0,덤덤씨 이야기,최덤덤,9.26,1.2만
1,우당탕탕 콩삼이,콩삼이,9.66,15만
2,진<zin>,나련수,8.42,"5,765"
3,아빠는 N살,유영근,9.90,"4,956만"
4,산군대잔치,다영좍가,9.87,464만
5,꽈따와 구갈이,똘정 X 르지,9.62,12만
6,타임머신 만들기,자드나인,9.49,3.7만
7,집에서 뽀짝뽀짝,그랑,9.67,16만
8,나태한 마왕은 신과 싸우기로 결심한다.,백창훈,9.87,150만
9,꿈꾸는 학회의 소문,이꼬,8.07,"1,589"


### 문제2
- 1 ~ 20 페이지까지 추출해서 데이터프레임으로 생성

In [6]:
# naver_webtoon_df_final
# naver_webtoon.csv

In [7]:
base_url = 'https://comic.naver.com/bestChallenge?page='

In [8]:
def get_df(url):
    driver.get(url)
    time.sleep(2)
    lis = driver.find_elements(By.CSS_SELECTOR, 'ul.BestChallengeView__challenge_list--sUqhh li')
    
    name, writer, rate, view = [[] for _ in range(4)]
    
    for li in lis:
        name.append(li.find_element(By.CLASS_NAME, 'ContentTitle__title--e3qXt').text)
        writer.append(li.find_element(By.CLASS_NAME, 'ContentAuthor__author--CTAAP').text)
        rate.append(li.find_element(By.CLASS_NAME, 'Rating__star_area--dFzsb').text.split('\n')[1])
        view.append(li.find_element(By.CLASS_NAME, 'Rating__view_area--GQb_S').text.split('\n')[1])
    
    df = pd.DataFrame({
        '제목' : name,
        '작가' : writer,
        '별점' : rate,
        '조회수' : view
    })

    return df

In [9]:
naver_webtoon_df = pd.DataFrame({
    '제목' : [],
    '작가' : [],
    '별점' : [],
    '조회수' : []
})

for i in range(1, 21):
    print('%2d번째 페이지 추출충'%i)
    url = base_url+str(i)
    df = get_df(url)
    naver_webtoon_df = pd.concat([naver_webtoon_df, df], axis=0, ignore_index=True)

naver_webtoon_df

 1번째 페이지 추출충
 2번째 페이지 추출충
 3번째 페이지 추출충
 4번째 페이지 추출충
 5번째 페이지 추출충
 6번째 페이지 추출충
 7번째 페이지 추출충
 8번째 페이지 추출충
 9번째 페이지 추출충
10번째 페이지 추출충
11번째 페이지 추출충
12번째 페이지 추출충
13번째 페이지 추출충
14번째 페이지 추출충
15번째 페이지 추출충
16번째 페이지 추출충
17번째 페이지 추출충
18번째 페이지 추출충
19번째 페이지 추출충
20번째 페이지 추출충


,제목,작가,별점,조회수
0,덤덤씨 이야기,최덤덤,9.26,1.2만
1,우당탕탕 콩삼이,콩삼이,9.66,15만
2,진<zin>,나련수,8.42,"5,765"
3,아빠는 N살,유영근,9.90,"4,956만"
4,산군대잔치,다영좍가,9.87,464만
...,...,...,...,...
595,호카헤이,박찬도,9.44,"2,203"
596,평범한 너,빅오링,9.00,"3,021"
597,헬레인저,오낭,9.49,"2,634"
598,이세계종말,이도,9.38,4.4만
